In this competition , we are asked to predict the toxicity rating of the comments taking into consideration the bias.Lets try it out.

In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import operator
import re
import time

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))


import gc
# Any results you write to the current directory are saved as output.

import warnings
warnings.filterwarnings('ignore')

['jigsaw-unintended-bias-in-toxicity-classification', 'fasttext-crawl-300d-2m']


In [9]:
%%time

train=pd.read_csv("../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv")
test=pd.read_csv("../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv")
submission=pd.read_csv("../input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv")

199610it [00:30, 13299.89it/s]

CPU times: user 11.2 s, sys: 1.21 s, total: 12.4 s
Wall time: 12.4 s


In [10]:
train.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.0,4,47


The column which we need to predict is the target and the comment_text contains all the comments in the forum.

In [11]:
test.head()

,id,comment_text
0,7000000,Jeff Sessions is another one of Trump's Orwell...
1,7000001,I actually inspected the infrastructure on Gra...
2,7000002,No it won't . That's just wishful thinking on ...
3,7000003,Instead of wringing our hands and nibbling the...
4,7000004,how many of you commenters have garbage piled ...


## Loading the embeddings

We use the fasttext embeddings here .All codes are borrowed/inspired from theo veils kernel -https://www.kaggle.com/theoviel/improve-your-score-with-some-text-preprocessing

In [12]:
EMB_PATH='../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec'

In [13]:
def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')

def load_embed(embed_dir=EMB_PATH):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in tqdm(open(embed_dir)))
    return embedding_index

def build_vocab(texts):
    sentences = texts.apply(lambda x: x.split()).values
    vocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

def check_coverage(vocab, embeddings_index):
    known_words = {}
    unknown_words = {}
    nb_known_words = 0
    nb_unknown_words = 0
    for word in vocab.keys():
        try:
            known_words[word] = embeddings_index[word]
            nb_known_words += vocab[word]
        except:
            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(known_words) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(nb_known_words / (nb_known_words + nb_unknown_words)))
    unknown_words = sorted(unknown_words.items(), key=operator.itemgetter(1))[::-1]

    return unknown_words


In [14]:
### Loading the embedding,

embed =load_embed()


0it [00:00, ?it/s]
1415it [00:00, 14148.02it/s]
2776it [00:00, 13980.91it/s]
4157it [00:00, 13928.50it/s]
5587it [00:00, 14037.63it/s]
6958it [00:00, 13935.81it/s]
8369it [00:00, 13985.50it/s]
9763it [00:00, 13971.34it/s]
11138it [00:00, 13904.07it/s]
12536it [00:00, 13925.97it/s]
13938it [00:01, 13952.67it/s]
15317it [00:01, 13903.30it/s]
16680it [00:01, 13655.08it/s]
18028it [00:01, 13350.14it/s]
19426it [00:01, 13532.83it/s]
20787it [00:01, 13554.02it/s]
22159it [00:01, 13602.09it/s]
23559it [00:01, 13718.22it/s]
24940it [00:01, 13744.34it/s]
26313it [00:01, 13646.09it/s]
27683it [00:02, 13659.24it/s]
29061it [00:02, 13694.73it/s]
30431it [00:02, 13658.85it/s]
31845it [00:02, 13798.40it/s]
33249it [00:02, 13869.37it/s]
34640it [00:02, 13879.60it/s]
36031it [00:02, 13888.56it/s]
37420it [00:02, 13858.12it/s]
38936it [00:02, 14224.27it/s]
40372it [00:02, 14263.53it/s]
41801it [00:03, 14241.38it/s]
43227it [00:03, 14218.58it/s]
44650it [00:03, 13976.36it/s]
46050it [00:03, 13960.37it/

336520it [00:24, 13464.68it/s]
337908it [00:24, 13586.64it/s]
339270it [00:24, 13592.34it/s]
340642it [00:24, 13629.79it/s]
342007it [00:25, 13372.85it/s]
343401it [00:25, 13537.21it/s]
344757it [00:25, 13517.22it/s]
346144it [00:25, 13620.29it/s]
347551it [00:25, 13750.84it/s]
348928it [00:25, 13232.33it/s]
350257it [00:25, 13000.96it/s]
351578it [00:25, 13061.39it/s]
352914it [00:25, 13147.54it/s]
354240it [00:25, 13178.47it/s]
355560it [00:26, 12845.48it/s]
356947it [00:26, 13135.30it/s]
358426it [00:26, 13590.24it/s]
359840it [00:26, 13749.28it/s]
361235it [00:26, 13807.45it/s]
362620it [00:26, 13228.56it/s]
363951it [00:26, 13156.41it/s]
365273it [00:26, 12594.89it/s]
366542it [00:26, 12249.61it/s]
367776it [00:27, 12052.81it/s]
368988it [00:27, 11976.09it/s]
370191it [00:27, 11587.14it/s]
371359it [00:27, 11612.80it/s]
372537it [00:27, 11659.73it/s]
373706it [00:27, 11567.34it/s]
374876it [00:27, 11605.11it/s]
376039it [00:27, 11284.95it/s]
377240it [00:27, 11491.08it/s]
378395it

644659it [00:49, 13357.62it/s]
646004it [00:49, 13383.85it/s]
647485it [00:49, 13780.39it/s]
648867it [00:49, 13510.69it/s]
650222it [00:49, 13298.06it/s]
651568it [00:49, 13344.13it/s]
652948it [00:49, 13476.69it/s]
654312it [00:49, 13525.02it/s]
655666it [00:50, 13099.35it/s]
657050it [00:50, 13313.06it/s]
658385it [00:50, 13324.11it/s]
659769it [00:50, 13472.97it/s]
661128it [00:50, 13506.71it/s]
662481it [00:50, 13177.62it/s]
663855it [00:50, 13340.51it/s]
665203it [00:50, 13378.71it/s]
666548it [00:50, 13397.94it/s]
667890it [00:50, 13396.11it/s]
669231it [00:51, 12877.93it/s]
670684it [00:51, 13330.66it/s]
672025it [00:51, 13218.42it/s]
673353it [00:51, 12666.87it/s]
674629it [00:51, 12341.32it/s]
675872it [00:51, 11639.09it/s]
677050it [00:51, 11583.13it/s]
678219it [00:51, 11557.55it/s]
679382it [00:51, 11469.48it/s]
680534it [00:52, 11448.00it/s]
681683it [00:52, 11396.62it/s]
682826it [00:52, 10853.27it/s]
683953it [00:52, 10971.89it/s]
685213it [00:52, 11414.20it/s]
686481it

961894it [01:13, 12819.44it/s]
963260it [01:13, 13059.63it/s]
964594it [01:14, 13141.90it/s]
965922it [01:14, 13181.45it/s]
967242it [01:14, 13157.92it/s]
968559it [01:14, 12892.03it/s]
969851it [01:14, 12713.80it/s]
971151it [01:14, 12796.36it/s]
972483it [01:14, 12948.88it/s]
973836it [01:14, 13117.82it/s]
975150it [01:14, 12864.54it/s]
976515it [01:15, 13090.10it/s]
977847it [01:15, 13157.94it/s]
979187it [01:15, 13228.66it/s]
980546it [01:15, 13333.69it/s]
981882it [01:15, 13213.12it/s]
983205it [01:15, 13147.16it/s]
984575it [01:15, 13307.15it/s]
985907it [01:15, 13172.26it/s]
987248it [01:15, 13241.65it/s]
988573it [01:15, 13177.15it/s]
989892it [01:16, 13077.83it/s]
991244it [01:16, 13204.31it/s]
992570it [01:16, 13218.96it/s]
993921it [01:16, 13302.99it/s]
995292it [01:16, 13421.96it/s]
996642it [01:16, 13442.54it/s]
997987it [01:16, 13426.39it/s]
999341it [01:16, 13458.27it/s]
1000691it [01:16, 13469.28it/s]
1002039it [01:16, 13337.55it/s]
1003374it [01:17, 13026.49it/s]
10047

1276879it [01:37, 13047.04it/s]
1278329it [01:37, 13451.35it/s]
1279701it [01:37, 13529.75it/s]
1281059it [01:38, 13455.00it/s]
1282408it [01:38, 13343.60it/s]
1283745it [01:38, 13146.70it/s]
1285090it [01:38, 13232.58it/s]
1286415it [01:38, 13156.48it/s]
1287749it [01:38, 13207.13it/s]
1289071it [01:38, 12811.97it/s]
1290472it [01:38, 13148.94it/s]
1291833it [01:38, 13280.91it/s]
1293165it [01:38, 13234.30it/s]
1294516it [01:39, 13313.80it/s]
1295850it [01:39, 12992.29it/s]
1297165it [01:39, 13037.65it/s]
1298553it [01:39, 13277.55it/s]
1299884it [01:39, 13264.55it/s]
1301259it [01:39, 13406.45it/s]
1302602it [01:39, 12930.48it/s]
1304090it [01:39, 13457.45it/s]
1305483it [01:39, 13595.51it/s]
1306852it [01:40, 13623.26it/s]
1308219it [01:40, 13458.63it/s]
1309569it [01:40, 13004.42it/s]
1310876it [01:40, 12939.26it/s]
1312231it [01:40, 13116.14it/s]
1313569it [01:40, 13192.05it/s]
1314913it [01:40, 13264.47it/s]
1316242it [01:40, 13138.98it/s]
1317558it [01:40, 12886.81it/s]
1318869i

1570044it [02:01, 13357.97it/s]
1571502it [02:01, 13701.27it/s]
1572877it [02:02, 13672.84it/s]
1574248it [02:02, 13634.78it/s]
1575614it [02:02, 13546.27it/s]
1576971it [02:02, 13551.17it/s]
1578328it [02:02, 13500.76it/s]
1579820it [02:02, 13895.02it/s]
1581213it [02:02, 13549.47it/s]
1582573it [02:02, 13558.51it/s]
1583932it [02:02, 13509.68it/s]
1585286it [02:02, 13408.90it/s]
1586629it [02:03, 13337.05it/s]
1587964it [02:03, 12848.20it/s]
1589300it [02:03, 12996.86it/s]
1590620it [02:03, 13055.17it/s]
1592083it [02:03, 13489.47it/s]
1593470it [02:03, 13599.19it/s]
1594834it [02:03, 13152.23it/s]
1596248it [02:03, 13433.73it/s]
1597694it [02:03, 13723.59it/s]
1599072it [02:03, 13700.68it/s]
1600454it [02:04, 13734.24it/s]
1601831it [02:04, 13253.25it/s]
1603189it [02:04, 13346.97it/s]
1604543it [02:04, 13403.22it/s]
1605887it [02:04, 13314.13it/s]
1607259it [02:04, 13429.27it/s]
1608628it [02:04, 13505.40it/s]
1609980it [02:04, 13137.98it/s]
1611343it [02:04, 13280.35it/s]
1612722i

1886385it [02:25, 13117.51it/s]
1887699it [02:25, 12769.98it/s]
1889145it [02:25, 13232.22it/s]
1890495it [02:26, 13308.93it/s]
1891831it [02:26, 13311.55it/s]
1893166it [02:26, 13311.67it/s]
1894500it [02:26, 13279.63it/s]
1895838it [02:26, 13308.68it/s]
1897264it [02:26, 13579.19it/s]
1898625it [02:26, 13553.55it/s]
1899982it [02:26, 13555.35it/s]
1901339it [02:26, 13132.46it/s]
1902733it [02:26, 13364.65it/s]
1904073it [02:27, 13363.42it/s]
1905413it [02:27, 13373.44it/s]
1906753it [02:27, 13315.01it/s]
1908185it [02:27, 13600.01it/s]
1909548it [02:27, 13314.80it/s]
1910883it [02:27, 13216.89it/s]
1912250it [02:27, 13348.93it/s]
1913691it [02:27, 13649.40it/s]
1915135it [02:27, 13822.02it/s]
1916520it [02:28, 13655.39it/s]
1917888it [02:28, 13457.26it/s]
1919236it [02:28, 13402.49it/s]
1920578it [02:28, 13339.81it/s]
1921914it [02:28, 13284.19it/s]
1923244it [02:28, 12857.14it/s]
1924601it [02:28, 13062.54it/s]
1925914it [02:28, 13081.33it/s]
1927240it [02:28, 13132.74it/s]
1928555i

In [15]:
# vocab=build_vocab(train['comment_text'])

In [16]:
# oov_fasttext=check_coverage(vocab,embed)

We see that we dont have embeddings for close to ~10 % of the text.Lets investigate it.

In [17]:
# oov_fasttext[:10]

We see that the contractions are creating trouble.This is because the fasttext does not understand contractions.Lets clean them.

In [18]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

In [19]:
def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

In [20]:
train['comment_text_lc']=train['comment_text'].apply(lambda x:x.lower())

In [21]:
train['comment_text_lc'].head()

0    this is so cool. it's like, 'would you want yo...
1    thank you!! this would make my life a lot less...
2    this is such an urgent design problem; kudos t...
3    is this something i'll be able to install on m...
4                 haha you guys are a bunch of losers.
Name: comment_text_lc, dtype: object

In [22]:
%%time

train['comment_text_lc']=train['comment_text_lc'].apply(lambda x:clean_contractions(x,contraction_mapping))

CPU times: user 22.6 s, sys: 420 ms, total: 23.1 s
Wall time: 23.1 s


In [23]:
# vocab=build_vocab(train['comment_text_lc'])

In [24]:
# oov_fasttext=check_coverage(vocab,embed)

There is only a marginal improvement in the embeddings list.

In [25]:
# oov_fasttext[:20]

There are also some special characters due to which we could not embed.Lets deal with them.

In [26]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'

In [27]:
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

In [28]:
def clean_special_charac(text,punc,mapping):
    for p in mapping:
        text=text.replace(p,mapping[p])
    for p in punc:
        text=text.replace(p,f' {p} ')
        
    return(text)

In [29]:
%%time

train['comment_treated']=train['comment_text_lc'].apply(lambda x:clean_special_charac(x,punct,punct_mapping))

CPU times: user 56.2 s, sys: 0 ns, total: 56.2 s
Wall time: 56.3 s


In [30]:
train[['comment_text_lc','comment_treated']].head(10)

,comment_text_lc,comment_treated
0,"this is so cool. it is like, 'would you want y...","this is so cool . it is like , ' would yo..."
1,thank you!! this would make my life a lot less...,thank you ! ! this would make my life a lot ...
2,this is such an urgent design problem; kudos t...,this is such an urgent design problem ; kudos...
3,is this something i will be able to install on...,is this something i will be able to install on...
4,haha you guys are a bunch of losers.,haha you guys are a bunch of losers .
5,ur a sh*tty comment.,ur a sh * tty comment .
6,hahahahahahahahhha suck it.,hahahahahahahahhha suck it .
7,ffffuuuuuuuuuuuuuuu,ffffuuuuuuuuuuuuuuu
8,the ranchers seem motivated by mostly by greed...,the ranchers seem motivated by mostly by greed...
9,it was a great show. not a combo i would of ex...,it was a great show . not a combo i would of ...


In [31]:
# vocab=build_vocab(train['comment_treated'])

In [32]:
# oov_fasttext=check_coverage(vocab,embed)

Thus we were able to get close to 99.5 % of the text for embedding.

In [33]:
# oov_fasttext[:10]

Lets complete the preprocessing step for test dataset aswell

In [34]:
%%time

test['comment_text_lc']=test['comment_text'].apply(lambda x:x.lower())
test['comment_text_lc']=test['comment_text_lc'].apply(lambda x:clean_contractions(x,contraction_mapping))
test['comment_treated']=test['comment_text_lc'].apply(lambda x:clean_special_charac(x,punct,punct_mapping))

CPU times: user 4.12 s, sys: 16 ms, total: 4.14 s
Wall time: 4.14 s


In [35]:
# vocab_test=build_vocab(test['comment_treated'])
# oof_fasttext_test=check_coverage(vocab_test,embed)

In [36]:
### save our model to temp file ,

train.to_csv("train_cleaned.csv",index=False)
test.to_csv("test_cleaned.csv",index=False)

Lets get to modelling  . Inspired from this kernel - https://www.kaggle.com/thousandvoices/simple-lstm

In [ ]:
# from keras.models import Model
# from keras.layers import Input, Dense, Embedding, SpatialDropout1D, Dropout, add, concatenate
# from keras.layers import CuDNNLSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
# from keras.preprocessing import text, sequence
# from keras.callbacks import LearningRateScheduler
# from sklearn.model_selection import train_test_split,StratifiedKFold
# from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping

In [ ]:
# del(vocab,vocab_test,oov_fasttext,oof_fasttext_test)
# gc.collect()

In [ ]:
# def build_matrix(word_index,embedding_index):
#     embedding_matrix = np.zeros((len(word_index) + 1, 300))
#     for word, i in word_index.items():
#         try:
#             embedding_matrix[i] = embedding_index[word]
#         except KeyError:
#             pass
#     return embedding_matrix

In [ ]:
# identity_columns = [
#     'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
#     'muslim', 'black', 'white', 'psychiatric_or_mental_illness']

# # Convert taget and identity columns to booleans
# def convert_to_bool(df, col_name):
#     df[col_name] = np.where(df[col_name] >= 0.5,1,0)
    
# def convert_dataframe_to_bool(df):
#     bool_df = df.copy()
#     for col in ['target'] + identity_columns:
#         print(f'\nConverting {col} to boolean')
#         convert_to_bool(bool_df, col)
#     return bool_df

# train = convert_dataframe_to_bool(train)

In [ ]:
# train.head()

In [ ]:
# #x_train=train['comment_treated']
# y_train=train['target']
# #y_aux_train = train[['target', 'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish','muslim', 'black', 'white', 'psychiatric_or_mental_illness']]
# #x_test=test['comment_treated']


In [ ]:
# print(f'Shape of x_train {x_train.shape} \n Shape of y_train {y_train.shape} \n Shape of x_test {x_test.shape}'
# )

In [ ]:
# tokenizer = text.Tokenizer()
# tokenizer.fit_on_texts(list(train['comment_treated']) + list(test['comment_treated']))


In [ ]:
# MAX_LEN=220
# LSTM_UNITS=128
# NUM_MODELS = 1
# BATCH_SIZE = 512
# DENSE_HIDDEN_UNITS = 2 * LSTM_UNITS
# EPOCHS = 5

In [ ]:
# embedding_matrix=build_matrix(tokenizer.word_index,embed)

In [ ]:
# ### Defining the metrics:
# ### https://www.kaggle.com/dborkan/benchmark-kernel/
# SUBGROUP_AUC = 'subgroup_auc'
# BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
# BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

# def compute_auc(y_true, y_pred):
#     try:
#         return metrics.roc_auc_score(y_true, y_pred)
#     except ValueError:
#         return np.nan

# def compute_subgroup_auc(df, subgroup, label, model_name):
#     subgroup_examples = df[df[subgroup]]
#     return compute_auc(subgroup_examples[label], subgroup_examples[model_name])

# def compute_bpsn_auc(df, subgroup, label, model_name):
#     """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
#     subgroup_negative_examples = df[df[subgroup] & ~df[label]]
#     non_subgroup_positive_examples = df[~df[subgroup] & df[label]]
#     examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
#     return compute_auc(examples[label], examples[model_name])

# def compute_bnsp_auc(df, subgroup, label, model_name):
#     """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
#     subgroup_positive_examples = df[df[subgroup] & df[label]]
#     non_subgroup_negative_examples = df[~df[subgroup] & ~df[label]]
#     examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
#     return compute_auc(examples[label], examples[model_name])

# def compute_bias_metrics_for_model(dataset,
#                                    subgroups,
#                                    model,
#                                    label_col,
#                                    include_asegs=False):
#     """Computes per-subgroup metrics for all subgroups and one model."""
#     records = []
#     for subgroup in subgroups:
#         record = {
#             'subgroup': subgroup,
#             'subgroup_size': len(dataset[dataset[subgroup]])
#         }
#         record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
#         record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
#         record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
#         records.append(record)
#     return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)




In [ ]:
# def calculate_overall_auc(df, model_name):
#     true_labels = df[TOXICITY_COLUMN]
#     predicted_labels = df[model_name]
#     return metrics.roc_auc_score(true_labels, predicted_labels)

# def power_mean(series, p):
#     total = sum(np.power(series, p))
#     return np.power(total / len(series), 1 / p)

# def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
#     bias_score = np.average([
#         power_mean(bias_df[SUBGROUP_AUC], POWER),
#         power_mean(bias_df[BPSN_AUC], POWER),
#         power_mean(bias_df[BNSP_AUC], POWER)
#     ])
#     return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)
    


In [ ]:
# def build_model(x_train,y_train,x_valid,y_valid,embedding_matrix,patience=3):
    
#     early_stop = EarlyStopping(monitor="val_loss", mode="min", patience=patience)
#     words = Input(shape=(MAX_LEN,))
#     x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words)
#     x = SpatialDropout1D(0.3)(x)
#     x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)

#     hidden = GlobalMaxPooling1D()(x)
#     hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='tanh')(hidden)])
#     result = Dense(1, activation='sigmoid')(hidden)
#     model = Model(inputs=words, outputs=result)
#     model.compile(loss='binary_crossentropy', optimizer='adam',metrics=["accuracy"])
#     model.fit(x_train,y_train,batch_size=128,epochs=3, validation_data=(x_valid, y_valid), 
#                         verbose=2, callbacks=[early_stop])

#     return model

In [ ]:
# n_fold = 5
# folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=11)

In [ ]:
# # valid_checkpoint_predictions = []
# # checkpoint_predictions=[]
# # valid_weights = []
# # weights=[]
# # for model_idx in range(NUM_MODELS):
# #     model = build_model(embedding_matrix, y_aux_train_df.shape[-1])
# #     for global_epoch in range(EPOCHS):
# #         model.fit(
# #             x_train_df,
# #             [y_train_df,y_aux_train_df],
# #             batch_size=BATCH_SIZE,
# #             epochs=1,
# #             verbose=2,
# #             callbacks=[
# #                 LearningRateScheduler(lambda epoch: 1e-3 * (0.6 ** global_epoch))
# #             ]
# #         )
# #         valid_checkpoint_predictions.append(model.predict(x_valid_df, batch_size=2048)[0].flatten())
# #         valid_weights.append(2 ** global_epoch)
# # checkpoint_predictions.append(model.predict(x_test,batch=2048)[0].flatten())


# # predictions = np.average(checkpoint_predictions, axis=0)

# def train_model(X, X_test, y):
    
#     oof = np.zeros((len(X), 1))
#     prediction = np.zeros((len(X_test), 1))
#     scores = []
#     for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y)):
#         print('Fold', fold_n, 'started at', time.ctime())
#         X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
#         y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
#         valid_df = X_valid.copy()    
#         print(f'\n Tokenizing for fold {fold_n}')
#         train_tokenised=tokenizer.texts_to_sequences(X_train['comment_treated'])
#         valid_tokenised=tokenizer.texts_to_sequences(X_valid['comment_treated'])
#         print(f'\n Tokenising completed.Starting padding for fold {fold_n}')
#         X_train=sequence.pad_sequences(train_tokenised,maxlen=MAX_LEN)
#         X_valid=sequence.pad_sequences(valid_tokenised,maxlen=MAX_LEN)
#         print(f'\n Padding completed .Started model building for fold {fold_n}')
#         model = build_model(X_train, y_train, X_valid, y_valid,embedding_matrix,patience=3)
#         print(f'\n Model building completed for fold {fold_n}')
#         pred_valid = model.predict(X_valid)
#         oof[valid_index] = pred_valid
#         valid_df[oof_name] = pred_valid
        
#         bias_metrics_df = compute_bias_metrics_for_model(valid_df, identity_columns, oof_name, 'target')
#         scores.append(get_final_metric(bias_metrics_df, calculate_overall_auc(valid_df, oof_name)))
        
#         prediction += model.predict(X_test, batch_size = 1024, verbose = 1)
    
#     prediction /= n_fold
    
#     # print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
#     return oof, prediction, scores

In [ ]:
# oof_name = 'predicted_target'
# oof, prediction, scores = train_model(X=train, X_test=test, y=y_train)
# print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))

In [ ]:
# submission = pd.DataFrame.from_dict({
#     'id': test['id'],
#     'prediction': predictions
# })

# submission.to_csv('submission.csv', index=False)

In [ ]:
# submission.head()